As we've written previously, our controller source code has to be written in a specific format i.e. in the *group/version/resource* structure. The actual reason behind this is because of the API Machinery library. It gives us the tooling to serialize go types via JSON, YAML, and Protobuf and we have to follow the library's specs so that it will know how to parse our custom resources over the wire. 

The *types.go* file is particularly important since it defines all your Kinds and their subtrcutures. For my PodDeleter Kind I might have a struct inside *types.go* that might look something like this.

In [ ]:
const (
    ActiveState  = "ACTIVE"
    DormantState = "DORMANT"
)

type PodDeleter struct {
    metav1.TypeMeta // remember that each Kubernetes Kind MUST have these two top fields!!
    metav1.ObjectMeta

    Spec PodDeleterSpec
    Status PodDeleterStatus
}

type PodDeleterSpec struct {
    DeleteAll bool // delete either the pod it detects or all pods in the current namespace after detecing a new one
}

type PodDeleterStatus struct {
    State string // this would take the value of either ActiveState or DormantState
                 // actually, idek if this would be doable since idk if client-go has 
                 // a way of retrieving all pods in a namespace
}

The reason we need all of our structs to embed TypeMeta and ObjectMeta is because that's is how API Machinery recognizes our struct and transforms it into JSON format which can be fed over the wire. *register.go* is the file that defines the group and version your package relates to. If we recall, this is also the file where we **register** our structs into the cluster's Scheme; this is where we declare the function AddToScheme() which adds the group, version, and Kinds to the scheme. *zz_generated.deepcopy.go* contains deep copy methods for all the types you defined in your package; it's also a generated file if you couldn't tell from the name. This method is necessary for your structs so they can implement the *runtime.Object* interface. When working with structures in your code base you have to import them. I don't know if this means that we have to make our structs and then upload them to github so that we can use it in a different package meant for our controller OR if it means we can write the controller in the same code base that our types and structs are defined. As a side note, the *ObjectMeta* struct that Kinds have includes a field which tells you about the owner of the resource! Another good note is that when you're using you're clientsets, keeping the namespace parameter empty ("") will indicate a cluster wide search. 